# Genome Annotation

Materials for the Genome Annotation a BRAKER & TSEBRA Genome Annotation workshop by Katharina Hoff (katharina.hoff@uni-greifswald.de).

Please find slides for an introductory talk on genome annotation (with BRAKER and TSEBRA) at [genome_annotation_2023.pdf](genome_annotation.pdf) (**ToDo** @KatharinaHoff: add slides!)

In the following, we will walk through the process of genome annotation on the example of a small proportion of the *Arabidopsis thaliana* genome.

## Repeat masking

Repetitive sequences are a huge problem for genome annotation. Some repeats only coincidentally look like protein-coding genes, others (such as transposases) are protein-coding genes, but we usually are not interested in any of these "repeat genes" when trying to find protein-coding genes in a novel genome. Thus, a genome should be repeat-masked prior gene prediction. 

Repeat masking is a resource and time-consuming step that is out of scope for this workshop. We recommend using RepeatModeler2 ([paper](https://doi.org/10.1073/pnas.1921046117), [software](https://www.repeatmasker.org/RepeatModeler/) ) to construct a species-specific repeat library and mask the genome with RepeatMasker (ideally, you will perform these computations on a node with >70 threads, in a place with very fast storage i/o, possibly using RAM instead of actual hard drive as a temporary file storage place):

```
T=72 # you need a large number of threads and fast i/o storage
# a toy data set to be used in this session is stored within the docker container
# if not done so, yet, we link that file into our current working directory
if ! [ -f genome.fa ]
then
    ln -s /opt/BRAKER/example/genome.fa genome.fa
fi
GENOME=genome.fa
DB=some_db_name_that_fits_to_species

BuildDatabase -name ${DB} ${GENOME}
RepeatModeler -database ${DB} -pa ${T} -LTRStruct
RepeatMasker -pa 72 -lib ${DB}-families.fa -xsmall ${GENOME}
```

This results in a file `${GENOME}.masked`. 

<details>
  <summary><b>Click to learn how to mask more rigorously when needed</b></summary>
Depending on the kind of genome, plenty of unmasked repeats may still persist. This is generally an issue to be expected in large genomes, such as vertebrate genomes, and you will notice the problem if the count of predicted proteins is extremely high. You can try to overcome "under-masking" with the following steps (we are suggesting to use GNU parallel to speed up the process):

```
ln -s genome.masked.fa genome.fa
splitMfasta.pl --minsize=25000000 ${GENOME}.masked

# Running TRF
ls genome.split.*.fa | parallel 'trf {} 2 7 7 80 10 50 500 -d -m -h'

# Parsing TRF output
# The script parseTrfOutput.py is from https://github.com/gatech-genemark/BRAKER2-exp
ls genome.split.*.fa.2.7.7.80.10.50.500.dat | parallel 'parseTrfOutput.py {} --minCopies 1 --statistics {}.STATS > {}.raw.gff 2> {}.parsedLog'

# Sorting parsed output..."
ls genome.split.*.fa.2.7.7.80.10.50.500.dat.raw.gff | parallel 'sort -k1,1 -k4,4n -k5,5n {} > {}.sorted 2> {}.sortLog'

# Merging gff...
FILES=genome.split.*.fa.2.7.7.80.10.50.500.dat.raw.gff.sorted
for f in $FILES
do
    bedtools merge -i $f | awk 'BEGIN{OFS="\t"} {print $1,"trf","repeat",$2+1,$3,".",".",".","."}' > $f.merged.gff 2> $f.bedtools_merge.log
done

# Masking FASTA chunk
ls genome.split.*.fa | parallel 'bedtools maskfasta -fi {} -bed {}.2.7.7.80.10.50.500.dat.raw.gff.sorted.merged.gff -fo {}.combined.masked -soft &> {}.bedools_mask.log'

# Concatenate split genome
cat genome.split.*.fa.combined.masked > genome.fa.combined.masked
```

The file `genome.fa.combined.masked` will be more rigorously masked.
</details>

## RNA-Seq alignment with HiSat2

Spliced alignments of RNA-Seq short reads are a valuable information source for predicting protein-coding genes with high accuracy.

<img src="et-rnaseq.png" alt="Figure 3 of Lomsadze et al. (2014) illustrates the use of RNA-Seq spliced alignments for predicting genes (with GeneMark-ET)." width="600"/>
Figure 3 of Lomsadze et al. (2014) illustrates the use of RNA-Seq spliced alignments for predicting genes (with GeneMark-ET, <a href=https://doi.org/10.1093/nar/gku55">Image Source</a>).

Executing HiSat2 is out of scope for the current session. You find a readily prepared alignment file in [/opt/BRAKER/example/RNAseq.bam](/opt/BRAKER/example/RNAseq.bam).

<details>
  <summary><b>If you want to see how such a file was prepared, click here and read.</b></summary>
  
We will map the *Arabidopsis thaliana* Illumina RNA-Seq reads from library SRR934391 in files [SRR934391_1.fastq.gz](/home/genomics/workshop_material/genome_annotation/sra/SRR934391_1.fastq.gz) and [SRR934391_2.fastq.gz](/home/genomics/workshop_material/genome_annotation/sra/SRR934391_2.fastq.gz). These are paired-end data, i.e. one file contains the forward reads while the other contains in the same order the reverse reads. The length of reads is in this case 100 nt.

We will use HiSat2 ([publication](https://doi.org/10.1038/s41587-019-0201-4), [software](https://github.com/DaehwanKimLab/hisat2)) to align these reads against a chunk of the *Arabidopsis thaliana* genome contained in the file [genome.fa](genome.fa). (You can in principle use any alignment tool capable of aligning RNA-seq reads to a genome, as long as it can perform spliced alignment.)

First, we need to build an index from the genome file:

```
# a toy data set to be used in this session is stored within the docker container
# if not done so, yet, we copy that file into our current working directory
if ! [ -f genome.fa ]
then
    ln -s /opt/BRAKER/example/genome.fa genome.fa
fi

# building the hisat2 index
hisat2-build genome.fa genome-idx 1> hisat2-build.log 2> hisat2-build.err
```

Inspect the log files [hisat2-build.log](hisat2-build.log) and [hisat2-build.err](hisat2-build.err) for possible errors.

Next, we align the RNA-seq reads against the genome. Consider to **not** do this on the Cesky Krumlov Workshop AWS resources. Performing this alignment took about 7 minutes with 70 threads. The precomputed output file is provided at `/home/genomics/workshop_material/genome_annotation/sra/SRR934391.sam`, and we will continue to use that pre-computed file.

```
T=4 # adjust to number of threads that you booted with

RNASEQDIR=/home/genomics/workshop_material/genome_annotation/sra/

time hisat2 -p ${T} -q -x genome-idx -1 ${RNASEQDIR}/SRR934391_1.fastq.gz \
    -2 ${RNASEQDIR}/SRR934391_2.fastq.gz -S rnaseq.sam \
    1> hisat2-align.log 2> hisat2-align.err
```

Our goal is to extract information on spliced alignments/intron positons from the alignment output file. To achieve this, we will use a tool called bam2hints that is part of the Augustus software suite ([software](https://github.com/Gaius-Augustus/Augustus)). However, this tool requires a sorted bam-file. Therefore, we first use Samtools ([paper](https://doi.org/10.1093/bioinformatics/btp352), [software](https://github.com/samtools) ) to convert the sam file to bam format:

```

T=4 # adjust to number of threads that you booted with, takes ~2 minutes with 4 threads

SAMFILE=/home/genomics/workshop_material/genome_annotation/sra/SRR934391.sam

time samtools view -@${T} -bSh ${SAMFILE} -o rnaseq.bam

# if you computed your own rnaseq.sam file, delete it to save space on harddrive
if [ -f rnaseq.sam ]
then
    rm rnaseq.sam
fi
```

Then, we sort that bam file (this will require a bit less than 4 GB of RAM):

```
T=4 # adjust to number of threads that you booted with, takes ~2 minutes with 4 threads

time samtools sort -@${T} -n rnaseq.bam -o rnaseq.s.bam

# remove the unsorted bam file to save space
rm rnaseq.bam
```

Careful, above bam file is just an demo example! We will be using a different bam file for running BRAKER because the above BAM file does not contain sufficient data for running BRAKER3, successfully!
</details>




## Annotation of protein coding genes

Structural genome annotation is ideally performed by a combination of a statistical model (e.g. Hidden Markov Model derivate) and extrinsic evidence (e.g. from transcriptomics or known protein sequences). The statistical model parameters have to be adapted to the genomic properties of novel species. For adapting parameters, an initial set of high-quality training genes from the target species is required. This is tricky to obtain. BRAKER is a perl script that comprises several pipelines to automated the solution of this problem: fully automatically generate an initial set of training genes, train gene finders, and then predict genes with the trained parameters and extrinsic evidence.

We will first take an approach to structural genome annotation that takes advantage both of RNA-Seq data, and a large database of known proteins, using BRAKER3 ([poster from PAG2023](https://www.researchgate.net/profile/Lars-Gabriel-3/publication/367409816_The_BRAKER3_Genome_Annotation_Pipeline/links/63d14cbae922c50e99c29c7a/The-BRAKER3-Genome-Annotation-Pipeline.pdf), [software](https://github.com/Gaius-Augustus/BRAKER)). If sufficient transcriptome data is available, then BRAKER3 is usually the best choice of pipeline. However, in the lack of transcriptome data, we need to consider alternative approaches. 

If transcriptome evidence is available but it just was not sufficient for obtaining good results with BRAKER3, then running BRAKER1 with RNA-Seq evidence ([paper](https://doi.org/10.1093/bioinformatics/btv661)) and protein supported gene prediction with BRAKER2 ([paper](https://doi.org/10.1093/nargab/lqaa108)), combined by TSEBRA ([paper](https://doi.org/10.1186/s12859-021-04482-0), [software](https://github.com/Gaius-Augustus/TSEBRA)) is often a good option.

In the total absence of transcriptome data, we recommend running either BRAKER2 with a large database of proteins, alone, or for larger genomes (such as vertebrates), we the recommend application of GALBA ([preprint](https://www.biorxiv.org/content/10.1101/2023.04.10.536199v1.abstract), [software](https://github.com/Gaius-Augustus/GALBA)) with reference proteomes of a few closely related, already annotated species.

### BRAKER3

BRAKER3 uses spliced aligned RNA-Seq data from Hisat2 ([paper](https://www.nature.com/articles/s41587-019-0201-4), [software](https://github.com/DaehwanKimLab/hisat2)) for genome-guided transcriptome assembly with Stringtie ([paper](https://www.nature.com/articles/nbt.3122), [software](https://github.com/gpertea/stringtie)). GeneMarkS-T ([paper](https://academic.oup.com/nar/article/43/12/e78/2902598), [software](http://exon.gatech.edu/genemark/license_download.cgi)) is used to call protein coding genes in the transcripts. These transcripts are "noisy", therefore, a large database of proteins (e.g. an OrthoDB partition) is used to filter these predictions, using among other GeneMark-specific scripts including ProtHint ([software](https://github.com/gatech-genemark/ProtHint)) with Spaln ([paper](https://academic.oup.com/bioinformatics/article/24/21/2438/191484), [software](https://github.com/ogotoh/spaln)) the fast search tool DIAMOND ([paper](https://www.nature.com/articles/s41592-021-01101-x), [software](https://github.com/bbuchfink/diamond)). Both, the transcriptome and protein evidence is then used by GeneMark-ETP ([preprint](https://www.biorxiv.org/content/10.1101/2023.01.13.524024v1), [software](https://github.com/gatech-genemark/GeneMark-ETP)) for self-training this HMM-based gene finder. This generates a training set for AUGUSTUS ([paper](https://doi.org/10.1093/bioinformatics/btn013), [software](https://github.com/Gaius-Augustus/Augustus)). Both, the GeneMark-ETP, and the AUGUSTUS gene set incorporate the evidence to some extent, and these gene sets are merged with TSEBRA by BRAKER3.

Training AUGUSTUS for a novel species usually comprises a step called etraining that adapts species-specific parameters of the statistical model of AUGUSTUS, and a step called optimize_augustus.pl that optimizes meta-parameters of that model. optimize_augustus.pl is very time-consuming, it yields usually ~2 percent points of accuracy on gene level. For this session, will disable this step with --skipOptimize. If you ever want to annotate a real new genome, make sure to delete `--skipOptimize` from your BRAKER calls (and expect substantially longer runtime). Also, GeneMark-ETP usually has a longer runtime. We will here set the maximal intergenic region for GeneMark-ETP to 10000. Please never apply this setting to a real genome annotation task, and expect a larger runtime.

In [17]:
%%script bash

T=4 # adjust to number of threads that you booted with, takes ~30 minutes with 4 threads

# delete output from a possible previous run if it exists
if [ -d BRAKER3 ]
then
    rm -rf BRAKER3
fi

ORTHODB=/home/genomics/workshop_material/genome_annotation/orthodb/Viridiplantae.fa # adjust to suitable clade

# run BRAKER3
time braker.pl --workingdir=BRAKER3 --genome=/opt/BRAKER/example/genome.fa --bam=/opt/BRAKER/example/RNAseq.bam \
    --prot_seq=${ORTHODB} --AUGUSTUS_BIN_PATH=/usr/bin/ \
    --AUGUSTUS_SCRIPTS_PATH=/usr/share/augustus/scripts/ --threads ${T} \
    --gm_max_intergenic 10000 --skipOptimize # remember to remove both these options for real jobs!
    # this call takes a few minutes even with --skipOptimize and --gm_max_intergenic 10000

#**********************************************************************************
#                               BRAKER CONFIGURATION                               
#**********************************************************************************
# BRAKER CALL: /opt/BRAKER/scripts/braker.pl --workingdir=BRAKER3 --genome=genome.fa --bam=/opt/BRAKER/example/RNAseq.bam --prot_seq=Viridiplantae.fa --AUGUSTUS_BIN_PATH=/usr/bin/ --AUGUSTUS_SCRIPTS_PATH=/usr/share/augustus/scripts/ --threads 4 --gm_max_intergenic 10000 --skipOptimize
# Mon May  1 20:35:40 2023: braker.pl version 3.0.3
# Mon May  1 20:35:40 2023:Both protein and RNA-Seq data in input detected. BRAKER will be executed in ETP mode (BRAKER3).
#*********
# Mon May  1 20:35:40 2023: Configuring of BRAKER for using external tools...
# Mon May  1 20:35:40 2023: Trying to set $AUGUSTUS_CONFIG_PATH...
# Mon May  1 20:35:40 2023: Found environment variable $AUGUSTUS_CONFIG_PATH.
# Mon May  1 20:35:40 2023: Checking /usr/share/augus

# Mon May  1 20:35:40 2023:Both protein and RNA-Seq data in input detected. BRAKER will be executed in ETP mode (BRAKER3).
#*********
# Mon May  1 20:35:40 2023: Log information is stored in file /home/katharina/git/GenomeAnnotation_Workshop2023/BRAKER3/braker.log


#*********
# WARNING: Detected whitespace in fasta header of file /home/katharina/git/GenomeAnnotation_Workshop2023/Viridiplantae.fa. This may later on cause problems! The pipeline will create a new file without spaces or "|" characters and a genome_header.map file to look up the old and new headers. This message will be suppressed from now on!
#*********


#*********
# WARNING: Number of reliable training genes is low (117). Recommended are at least 600 genes
#*********



real	23m55.205s
user	137m12.708s
sys	1m47.704s


<details>
  <summary><b>Out of time, job died? Click here.</b></summary>
If you ran out of time (the BRAKER3 job takes substantial time), you may copy the most important files as follows from a notebook cell:

```
%%script bash
cp -r BRAKER3_precomputed_results BRAKER3
```
</details>

Let's inspect the output, the most important files are braker.gtf, Augustus/augustus.hints.gtf, and GeneMark-ETP/genemark.gtf:

In [1]:
%%script bash
cd BRAKER3
ls -lh braker.gtf Augustus/augustus.hints.gtf GeneMark-ETP/genemark.gtf

-rw-rw-r-- 1 katharina katharina 400K May  1 20:59 Augustus/augustus.hints.gtf
-rw-rw-r-- 1 katharina katharina 409K May  1 20:59 braker.gtf
-rw-rw-r-- 1 katharina katharina 407K May  1 20:59 GeneMark-ETP/genemark.gtf


The file [BRAKER3/what-to-cite.txt](BRAKER3/what-to-cite.txt) advises you on what papers should be cited if you were going to publish a manuscript on a gene set produced with BRAKER1.

Usually, the braker.gtf is the main output. However, because of the way that TSEBRA combines the augustus.hints.gtf and the genemark.gtf file with the hintsfile.gff, this may not be the optimal gene set. We can investigate this by descriptive statistics of gene sets (e.g.~how many genes are in a gene set, how many transcripts, how many exons does an average transcript have?), or we measure sensivity according to a conserved gene set, e.g. BUSCO or OMArk. Here, we will use BUSCO. Before running BUSCO, we need to make sure that we have protein sequences of all three gene sets (only the braker.aa exists by default):

In [3]:
%%script bash
# generate protein (and coding seq file) from AUGUSTUS predictions
cd BRAKER3/Augustus
getAnnoFastaFromJoingenes.py -g /opt/BRAKER/example/genome.fa -o augustus.hints -f augustus.hints.gtf
# generate protein (and coding seq file) from GeneMark-ETP predictions
cd ../GeneMark-ETP
getAnnoFastaFromJoingenes.py -g /opt/BRAKER/example/genome.fa -o genemark -f genemark.gtf
# see file sizes
cd ../
ls -lh braker.aa GeneMark-ETP/genemark.aa Augustus/augustus.hints.aa
# Count number of transcripts by counting FASTA headers
echo "Counting number of protein sequences = transcripts"
grep -c ">" braker.aa GeneMark-ETP/genemark.aa Augustus/augustus.hints.aa

-rw-r--r-- 1 katharina katharina 126K May  3 12:59 Augustus/augustus.hints.aa
-rw-r--r-- 1 katharina katharina 128K May  3 12:59 braker.aa
-rw-r--r-- 1 katharina katharina 116K May  3 12:59 GeneMark-ETP/genemark.aa
Counting number of protein sequences = transcripts
braker.aa:301
GeneMark-ETP/genemark.aa:278
Augustus/augustus.hints.aa:292


GALBA has a simple script to compute the ratio of mono- to multi-exonic genes (only counting one isoform if one gene has several alternative isoforms, that's why the transcript number differs from the number above for methods that contain alternative transcripts, such as AUGUSTUS and BRAKER):

In [4]:
%%script bash
cd BRAKER3
echo "Computing some descriptive statistics for BRAKER:"
analyze_exons.py -f braker.gtf
echo ""
echo "Doing the same for Augustus:"
analyze_exons.py -f Augustus/augustus.hints.gtf
echo ""
echo "And for GeneMark-ETP:"
analyze_exons.py -f GeneMark-ETP/genemark.gtf

Computing some descriptive statistics for BRAKER:
Number of transcripts: 286
Largest number of exons in all transcripts: 30
Monoexonic transcripts: 76
Multiexonic transcripts: 210
Mono:Mult Ratio: 0.36
Boxplot of number of exons per transcript:
Min: 1
25%: 1
50%: 4
75%: 7
Max: 30

Doing the same for Augustus:
Number of transcripts: 276
Largest number of exons in all transcripts: 30
Monoexonic transcripts: 70
Multiexonic transcripts: 206
Mono:Mult Ratio: 0.34
Boxplot of number of exons per transcript:
Min: 1
25%: 1
50%: 4
75%: 7
Max: 30

And for GeneMark-ETP:
Number of transcripts: 278
Largest number of exons in all transcripts: 30
Monoexonic transcripts: 66
Multiexonic transcripts: 212
Mono:Mult Ratio: 0.31
Boxplot of number of exons per transcript:
Min: 1
25%: 2
50%: 4
75%: 8
Max: 30


#### BUSCO assessment

BUSCO ([paper](https://doi.org/10.1002/cpz1.323), [software](https://gitlab.com/ezlab/busco)) can provide information on sensitivity with respect to a clade-specific core gene set. We will in the following use BUSCO to compare sensitivity in the BRAKER3, AUGUSTUS, and GeneMark-ETP gene set.

First, we find the closest BUSCO lineage (we are working on *Arabidopsis thaliana*):

In [5]:
%%script bash

source conda_init
conda activate busco_env

busco --list-datasets > busco_lineages.txt 2> busco_lineages.log

All available lineages are now in [busco_lineages.txt](busco_lineages.txt). (Check [busco_lineages.log](busco_lineages.log) for possible errors.)

Check at [NCBI taxonomy](https://www.ncbi.nlm.nih.gov/taxonomy) the lineage of the target *Arabidopsis*. I believe the lineage is:

`cellular organisms; Eukaryota; Viridiplantae; Streptophyta; Streptophytina; Embryophyta; Tracheophyta; Euphyllophyta; Spermatophyta; Magnoliopsida; Mesangiospermae; eudicotyledons; Gunneridae; Pentapetalae; rosids; malvids; Brassicales; Brassicaceae; Camelineae`

Now find a related lineage in [busco_lineages.txt](busco_lineages.txt). `brassicales_odb10` is the closest lineage.

Next, we run a BUSCO assessment on all gene sets (this takes ~3 minutes with 4 threads):

In [6]:
%%script bash

T=4 # adjust to number of threads that you booted with

source conda_init
conda activate busco_env

cd BRAKER3
# create links if not already present
if [ ! -L augustus.aa ]
then
    ln -s Augustus/augustus.hints.aa augustus.aa
fi

if [ ! -L genemark.aa ]
then
    ln -s GeneMark-ETP/genemark.aa genemark.aa
fi

GENESETS=(braker augustus genemark)

for g in ${GENESETS[@]}; do
    echo "Processing ${g}..."
    # delete old output if existing
    if [ -d busco_${g} ]
    then
        rm -r busco_${g}
    fi
    # run BUSCO
    busco -m proteins -i ${g}.aa -o busco_${g} \
        -l brassicales_odb10 -c ${T} &> busco_${g}.log
done

Processing braker...
Processing augustus...
Processing genemark...


Next, we visualize the BUSCO results:

In [7]:
%%script bash

source conda_init
conda activate busco_env

cd BRAKER3

# create BUSCO_summaries folder if not present
if ! [ -d BUSCO_summaries ]
then
    mkdir BUSCO_summaries
fi

# copy all BUSCO results into the summaries folder
cp busco_*/short_summary*.txt BUSCO_summaries

# generate BUSCO plot
generate_plot.py -wd BUSCO_summaries &> generate_plot.log

Check the file [generate_plot.log](generate_plot.log) for possible errors. This results in the following figure (stored at [BRAKER3/BUSCO_summaries/busco_figure.png](BRAKER3/BUSCO_summaries/busco_figure.png)):

<img src="BRAKER3/BUSCO_summaries/busco_figure.png" alt="BUSCO results" width="400"/>

The data that we used in this session was selected purely on the criterion of feasible runtime. In a real scenario, with a complete genome, the BUSCO plot should contain a much larger number of complete BUSCOs, and you are usually happy if the number BUSCOs in the BRAKER3 gene set is higher or equal to the number of BUSCOs detected in the AUGUSTUS and GeneMark-ETP set, while the total number of transcripts does not grow into an unexpected way (e.g. having 80.000 proteins in a BRAKER gene set does seem odd...).

But what can we do if there is no RNA-Seq data for a particular species? In that case, we can resort to using either BRAKER2 (for small and medium sized genomes, with a large database of proteins that might be only remotely related), or we may use GALBA (for large vertebrate genomes, with a few closely related reference proteomes).

### BRAKER2

BRAKER2 ([paper](https://doi.org/10.1093/nargab/lqaa108)) uses spliced alignment information from a huge database of proteins against the target genome. We typically use OrthoDB partitions of clades, hosted at https://bioinf.uni-greifswald.de/bioinf/partitioned_odb11/. Note: a set of proteins from one or a few related species is not sufficient for running BRAKER2. A particular set of proteins of a closely related species can be appended to a larger database for running BRAKER2. However, BRAKER2 is not an ideal tool for recovering a complete set of proteins from a related species.

The following call of BRAKER2 takes ~15 minutes on 4 threads, even when optimizing AUGUSTUS parameters is disabled:

In [2]:
%%script bash

T=4 # adjust to number of threads that you booted with

ORTHODB=/home/genomics/workshop_material/genome_annotation/orthodb/Viridiplantae.fa # adjust to suitable clade

# delete output from a possible previous run if it exists
if [ -d BRAKER2 ]
then
    rm -rf BRAKER2
fi

time braker.pl --workingdir=BRAKER2 --genome=/opt/BRAKER/example/genome.fa --prot_seq=${ORTHODB} \
    --AUGUSTUS_BIN_PATH=/usr/bin/ --AUGUSTUS_SCRIPTS_PATH=/usr/share/augustus/scripts/ --threads ${T} \
    --gm_max_intergenic 10000 --skipOptimize \ # remember to remove both options if you are running a real job
    2> braker2.log

#**********************************************************************************
#                               BRAKER CONFIGURATION                               
#**********************************************************************************
# BRAKER CALL: /opt/BRAKER/scripts/braker.pl --workingdir=BRAKER2 --genome=genome.fa --prot_seq=/home/genomics/workshop_material/genome_annotation/orthodb/Viridiplantae.fa --AUGUSTUS_BIN_PATH=/usr/bin/ --AUGUSTUS_SCRIPTS_PATH=/usr/share/augustus/scripts/ --threads 4 --gm_max_intergenic 10000 --skipOptimize  # remember to remove both options if you are running a real job
# Wed May  3 14:11:16 2023: braker.pl version 3.0.3
# Wed May  3 14:11:16 2023: Creating directory /home/katharina/git/GenomeAnnotation_Workshop2023/BRAKER2.
# Wed May  3 14:11:16 2023: Only Protein input detected, BRAKER will be executed in EP mode (BRAKER2).
# Wed May  3 14:11:16 2023: Configuring of BRAKER for using external tools...
# Wed May  3 14:11:16 2023: Trying t

# Wed May  3 14:11:16 2023: Log information is stored in file /home/katharina/git/GenomeAnnotation_Workshop2023/BRAKER2/braker.log


#*********
# WARNING: Detected whitespace in fasta header of file /home/genomics/workshop_material/genome_annotation/orthodb/Viridiplantae.fa. This may later on cause problems! The pipeline will create a new file without spaces or "|" characters and a genome_header.map file to look up the old and new headers. This message will be suppressed from now on!
#*********
ProtHint Version 2.6.0
Copyright 2019, Georgia Institute of Technology, USA

Please cite
  - ProtHint: https://doi.org/10.1093/nargab/lqaa026
  - DIAMOND:  https://doi.org/10.1038/nmeth.3176
  - Spaln:    https://doi.org/10.1093/bioinformatics/btn460

Called from: /home/katharina/git/GenomeAnnotation_Workshop2023/BRAKER2
Cmd: /opt/ETP/bin/gmes/ProtHint/bin/prothint.py --threads=4 --geneMarkGtf /home/katharina/git/GenomeAnnotation_Workshop2023/BRAKER2/GeneMark-ES/genemark.gtf /home/katharina/git/GenomeAnnotation_Workshop2023/BRAKER2/genome.fa /home/katharina/git/GenomeAnnotation_Workshop2023/BRAKER2/proteins.fa

[Wed May  3 14

[Wed May  3 14:17:37 2023] Enqueueing pair 334/6055 (5.5%)
[Wed May  3 14:17:37 2023] Enqueueing pair 340/6055 (5.6%)
[Wed May  3 14:17:37 2023] Enqueueing pair 346/6055 (5.7%)
[Wed May  3 14:17:37 2023] Enqueueing pair 352/6055 (5.8%)
[Wed May  3 14:17:37 2023] Enqueueing pair 358/6055 (5.9%)
[Wed May  3 14:17:37 2023] Enqueueing pair 364/6055 (6.0%)
[Wed May  3 14:17:37 2023] Enqueueing pair 370/6055 (6.1%)
[Wed May  3 14:17:37 2023] Enqueueing pair 376/6055 (6.2%)
[Wed May  3 14:17:37 2023] Enqueueing pair 382/6055 (6.3%)
[Wed May  3 14:17:37 2023] Enqueueing pair 388/6055 (6.4%)
[Wed May  3 14:17:37 2023] Enqueueing pair 394/6055 (6.5%)
[Wed May  3 14:17:37 2023] Enqueueing pair 400/6055 (6.6%)
[Wed May  3 14:17:37 2023] Enqueueing pair 406/6055 (6.7%)
[Wed May  3 14:17:37 2023] Enqueueing pair 412/6055 (6.8%)
[Wed May  3 14:17:37 2023] Enqueueing pair 418/6055 (6.9%)
[Wed May  3 14:17:37 2023] Enqueueing pair 424/6055 (7.0%)
[Wed May  3 14:17:37 2023] Enqueueing pair 430/6055 (7.1

[Wed May  3 14:17:37 2023] Enqueueing pair 1163/6055 (19.2%)
[Wed May  3 14:17:37 2023] Enqueueing pair 1169/6055 (19.3%)
[Wed May  3 14:17:37 2023] Enqueueing pair 1175/6055 (19.4%)
[Wed May  3 14:17:37 2023] Enqueueing pair 1181/6055 (19.5%)
[Wed May  3 14:17:37 2023] Enqueueing pair 1187/6055 (19.6%)
[Wed May  3 14:17:37 2023] Enqueueing pair 1193/6055 (19.7%)
[Wed May  3 14:17:37 2023] Enqueueing pair 1199/6055 (19.8%)
[Wed May  3 14:17:37 2023] Enqueueing pair 1205/6055 (19.9%)
[Wed May  3 14:17:37 2023] Enqueueing pair 1211/6055 (20.0%)
[Wed May  3 14:17:37 2023] Enqueueing pair 1218/6055 (20.1%)
[Wed May  3 14:17:37 2023] Enqueueing pair 1224/6055 (20.2%)
[Wed May  3 14:17:37 2023] Enqueueing pair 1230/6055 (20.3%)
[Wed May  3 14:17:37 2023] Enqueueing pair 1236/6055 (20.4%)
[Wed May  3 14:17:37 2023] Enqueueing pair 1242/6055 (20.5%)
[Wed May  3 14:17:37 2023] Enqueueing pair 1248/6055 (20.6%)
[Wed May  3 14:17:37 2023] Enqueueing pair 1254/6055 (20.7%)
[Wed May  3 14:17:37 202

...

[Wed May  3 14:25:03 2023] Enqueueing pair 2525/4200 (60.1%)
[Wed May  3 14:25:03 2023] Enqueueing pair 2529/4200 (60.2%)
[Wed May  3 14:25:03 2023] Enqueueing pair 2533/4200 (60.3%)
[Wed May  3 14:25:03 2023] Enqueueing pair 2537/4200 (60.4%)
[Wed May  3 14:25:03 2023] Enqueueing pair 2541/4200 (60.5%)
[Wed May  3 14:25:03 2023] Enqueueing pair 2546/4200 (60.6%)
[Wed May  3 14:25:03 2023] Enqueueing pair 2550/4200 (60.7%)
[Wed May  3 14:25:03 2023] Enqueueing pair 2554/4200 (60.8%)
[Wed May  3 14:25:03 2023] Enqueueing pair 2558/4200 (60.9%)
[Wed May  3 14:25:03 2023] Enqueueing pair 2562/4200 (61.0%)
[Wed May  3 14:25:03 2023] Enqueueing pair 2567/4200 (61.1%)
[Wed May  3 14:25:03 2023] Enqueueing pair 2571/4200 (61.2%)
[Wed May  3 14:25:03 2023] Enqueueing pair 2575/4200 (61.3%)
[Wed May  3 14:25:03 2023] Enqueueing pair 2579/4200 (61.4%)
[Wed May  3 14:25:03 2023] Enqueueing pair 2583/4200 (61.5%)
[Wed May  3 14:25:03 2023] Enqueueing pair 2588/4200 (61.6%)
[Wed May  3 14:25:03 202

[Wed May  3 14:25:07 2023] Enqueueing pair 3092/4200 (73.6%)
[Wed May  3 14:25:07 2023] Enqueueing pair 3096/4200 (73.7%)
[Wed May  3 14:25:09 2023] Enqueueing pair 3100/4200 (73.8%)
[Wed May  3 14:25:09 2023] Enqueueing pair 3104/4200 (73.9%)
[Wed May  3 14:25:09 2023] Enqueueing pair 3108/4200 (74.0%)
[Wed May  3 14:25:09 2023] Enqueueing pair 3113/4200 (74.1%)
[Wed May  3 14:25:09 2023] Enqueueing pair 3117/4200 (74.2%)
[Wed May  3 14:25:09 2023] Enqueueing pair 3121/4200 (74.3%)
[Wed May  3 14:25:09 2023] Enqueueing pair 3125/4200 (74.4%)
[Wed May  3 14:25:09 2023] Enqueueing pair 3129/4200 (74.5%)
[Wed May  3 14:25:09 2023] Enqueueing pair 3134/4200 (74.6%)
[Wed May  3 14:25:09 2023] Enqueueing pair 3138/4200 (74.7%)
[Wed May  3 14:25:09 2023] Enqueueing pair 3142/4200 (74.8%)
[Wed May  3 14:25:09 2023] Enqueueing pair 3146/4200 (74.9%)
[Wed May  3 14:25:09 2023] Enqueueing pair 3150/4200 (75.0%)
[Wed May  3 14:25:09 2023] Enqueueing pair 3155/4200 (75.1%)
[Wed May  3 14:25:09 202

[Wed May  3 14:25:11 2023] Enqueueing pair 3486/4200 (83.0%). Est. time left: 00:00:05 (hh:mm:ss)
[Wed May  3 14:25:11 2023] Enqueueing pair 3491/4200 (83.1%). Est. time left: 00:00:05 (hh:mm:ss)
[Wed May  3 14:25:11 2023] Enqueueing pair 3495/4200 (83.2%). Est. time left: 00:00:05 (hh:mm:ss)
[Wed May  3 14:25:11 2023] Enqueueing pair 3499/4200 (83.3%). Est. time left: 00:00:05 (hh:mm:ss)
[Wed May  3 14:25:12 2023] Enqueueing pair 3503/4200 (83.4%). Est. time left: 00:00:05 (hh:mm:ss)
[Wed May  3 14:25:12 2023] Enqueueing pair 3507/4200 (83.5%). Est. time left: 00:00:05 (hh:mm:ss)
[Wed May  3 14:25:12 2023] Enqueueing pair 3512/4200 (83.6%). Est. time left: 00:00:05 (hh:mm:ss)
[Wed May  3 14:25:12 2023] Enqueueing pair 3516/4200 (83.7%). Est. time left: 00:00:05 (hh:mm:ss)
[Wed May  3 14:25:12 2023] Enqueueing pair 3520/4200 (83.8%). Est. time left: 00:00:05 (hh:mm:ss)
[Wed May  3 14:25:12 2023] Enqueueing pair 3524/4200 (83.9%). Est. time left: 00:00:05 (hh:mm:ss)
[Wed May  3 14:25:12

[Wed May  3 14:25:14 2023] Enqueueing pair 3839/4200 (91.4%). Est. time left: 00:00:03 (hh:mm:ss)
[Wed May  3 14:25:14 2023] Enqueueing pair 3843/4200 (91.5%). Est. time left: 00:00:03 (hh:mm:ss)
[Wed May  3 14:25:14 2023] Enqueueing pair 3848/4200 (91.6%). Est. time left: 00:00:03 (hh:mm:ss)
[Wed May  3 14:25:14 2023] Enqueueing pair 3852/4200 (91.7%). Est. time left: 00:00:03 (hh:mm:ss)
[Wed May  3 14:25:14 2023] Enqueueing pair 3856/4200 (91.8%). Est. time left: 00:00:03 (hh:mm:ss)
[Wed May  3 14:25:14 2023] Enqueueing pair 3860/4200 (91.9%). Est. time left: 00:00:03 (hh:mm:ss)
[Wed May  3 14:25:14 2023] Enqueueing pair 3864/4200 (92.0%). Est. time left: 00:00:03 (hh:mm:ss)
[Wed May  3 14:25:14 2023] Enqueueing pair 3869/4200 (92.1%). Est. time left: 00:00:03 (hh:mm:ss)
[Wed May  3 14:25:14 2023] Enqueueing pair 3873/4200 (92.2%). Est. time left: 00:00:03 (hh:mm:ss)
[Wed May  3 14:25:14 2023] Enqueueing pair 3877/4200 (92.3%). Est. time left: 00:00:03 (hh:mm:ss)
[Wed May  3 14:25:14

[Wed May  3 14:25:17 2023] Enqueueing pair 4192/4200 (99.8%). Est. time left: 00:00:01 (hh:mm:ss)
[Wed May  3 14:25:17 2023] Enqueueing pair 4196/4200 (99.9%). Est. time left: 00:00:01 (hh:mm:ss)
[Wed May  3 14:25:30 2023] 4200/4200 (100%) pairs aligned
[Wed May  3 14:25:30 2023] Alignment of pairs finished
[Wed May  3 14:25:30 2023] Translating coordinates from local pair level to contig level
[Wed May  3 14:25:30 2023] Finished spliced alignment
[Wed May  3 14:25:30 2023] Flagging top chains
[Wed May  3 14:25:31 2023] Processing the output
[Wed May  3 14:25:33 2023] Output processed
[Wed May  3 14:25:33 2023] ProtHint finished.

real	14m50.860s
user	38m24.422s
sys	1m34.607s


<details>
  <summary><b>Out of time, job died? Click here.</b></summary>
If you ran out of time (the BRAKER2 job takes substantial time), you may copy the most important files as follows from a notebook cell:

```
%%script bash
cp -r BRAKER2_precomputed_results BRAKER2
```
</details>

The most important output files are:

   * [BRAKER2/braker.gtf](BRAKER2/braker.gtf)
   * [BRAKER2/Augustus/augustus.hints.gtf](BRAKER2/Augustus/augustus.hints.gtf)
   * [BRAKER2/GeneMark-EP/genemark.gtf](BRAKER2/GeneMark-EP/genemark.gtf)
   * [BRAKER2/hintsfile.gff](BRAKER2/hintsfile.gff)
   
The file [BRAKER2/what-to-cite.txt](BRAKER2/what-to-cite.txt) advises you on what papers should be cited if you were going to publish a manuscript on a gene set produced with BRAKER2. 

All methods described for BRAKER3 (BUSCO, number of transcripts, mono:mult exon ratio, etc.) are of course applicable to BRAKER2, GALBA, and BRAKER1, as well. We will skip it here because of time constraints.

### GALBA

GALBA ([preprint](https://www.biorxiv.org/content/10.1101/2023.04.10.536199v1.abstract), [software](https://github.com/Gaius-Augustus/GALBA)) is a BRAKER-spinoff that uses miniprot ([paper](https://doi.org/10.1093/bioinformatics/btad014), [software](https://github.com/lh3/miniprot)) to generate a training gene set of AUGUSTUS. In contrast to the BRAKER2 and BRAKER3 pipelines, GALBA is not very good at using remotely related protein evidence. However, given reference proteins of several closely related species, GALBA is very good at recovering gene structures, particularly in large vertebrate genomes. You may execute GALBA as follows (using a toy example data set, it executes within a 3 minutes on 4 threads):

In [1]:
%%script bash

T=4 # adjust to number of threads that you booted with

# delete output from a possible previous run if it exists
if [ -d GALBA ]
then
    rm -rf GALBA
fi

time galba.pl --workingdir=GALBA --genome=/opt/BRAKER/example/genome.fa --prot_seq=/opt/GALBA/example/proteins.fa \
    --AUGUSTUS_BIN_PATH=/usr/bin/ --AUGUSTUS_SCRIPTS_PATH=/usr/share/augustus/scripts/ --threads ${T} \
    --skipOptimize \ # remember to remove both options if you are running a real job
    2> galba.log

# Wed May  3 14:42:46 2023: Log information is stored in file /home/katharina/git/GenomeAnnotation_Workshop2023/GALBA/GALBA.log


info: Low coverage detected, coverage will be ignored in the high-confidence set.


#*********
# WARNING: Number of reliable training genes is low (226). Recommended are at least 600 genes
#*********
#*********
# WARNING: Number of reliable training genes is low (226). Recommended are at least 600 genes
#*********



real	2m46.563s
user	2m47.582s
sys	0m5.331s


<details>
  <summary><b>Out of time, job died? Click here.</b></summary>
It is unlikely that GALBA will note complete, fast. However, you may copy the most important files as follows from a notebook cell:

```
%%script bash
cp -r GALBA_precomputed_results GALBA
```
</details>

The most important output files are:

   * [GALBA/augustus.hints.gtf](GALBA/augustus.hints.gtf)
   * [GALBA/hintsfile.gff](GALBA/hintsfile.gff)
   
The file [GALBA/what-to-cite.txt](GALBA/what-to-cite.txt) advises you on what papers should be cited if you were going to publish a manuscript on a gene set produced with GALBA.

### BRAKER1

Since BRAKER3, the pipeline for input of both RNA-Seq and a large database of proteins achieves usually higher accuracy than BRAKER1 with RNA-Seq, only, BRAKER1 is now rather a pipeline that we may resort to using if BRAKER3 died due to insufficient data. BRAKER1 also requires a certain amount of RNA-Seq alignments but that is less than what is required for transcriptome assembly with StringTie in BRAKER3.

BRAKER1 uses spliced alignment information from RNA-Seq for training GeneMark-ET ([paper](https://doi.org/10.1093/nar/gku557), [software](http://exon.gatech.edu/genemark/license_download.cgi)), for selecting a training gene set for AUGUSTUS, and for predicting genes with AUGUSTUS. 

We will run BRAKER1 to predict genes in the genomic sequence with the prepared RNA-Seq intron evidence. As before, we introduce options to save runtime (see BRAKER3 and BRAKER2) that should not be applied in a real-life annotation project.

In [2]:
%%script bash

T=4 # adjust to number of threads that you booted with, takes ~2.5 minutes on 4 threads

# delete output from a possible previous run if it exists
if [ -d BRAKER1 ]
then
    rm -rf BRAKER1
fi

time braker.pl --workingdir=BRAKER1 --genome=/opt/BRAKER/example/genome.fa --bam=/opt/BRAKER/example/RNAseq.bam --softmasking \
    --AUGUSTUS_BIN_PATH=/usr/bin/ --AUGUSTUS_SCRIPTS_PATH=/usr/share/augustus/scripts/ --threads ${T} \
    --gm_max_intergenic 10000 --skipOptimize #  remember to remove this option if you are running a real job
    # this call takes a few minutes even with --skipOptimize

#**********************************************************************************
#                               BRAKER CONFIGURATION                               
#**********************************************************************************
# BRAKER CALL: /opt/BRAKER/scripts/braker.pl --workingdir=BRAKER1 --genome=genome.fa --bam=/opt/BRAKER/example/RNAseq.bam --softmasking --AUGUSTUS_BIN_PATH=/usr/bin/ --AUGUSTUS_SCRIPTS_PATH=/usr/share/augustus/scripts/ --threads 4 --gm_max_intergenic 10000 --skipOptimize
# Wed May  3 15:18:18 2023: braker.pl version 3.0.3
# Wed May  3 15:18:18 2023: Creating directory /home/katharina/git/GenomeAnnotation_Workshop2023/BRAKER1.
# Wed May  3 15:18:18 2023: Only RNA-Seq input detected, BRAKER will be executed in ET mode (BRAKER1).
# Wed May  3 15:18:18 2023: Configuring of BRAKER for using external tools...
# Wed May  3 15:18:18 2023: Trying to set $AUGUSTUS_CONFIG_PATH...
# Wed May  3 15:18:18 2023: Found environment variable $AUGUSTUS_CONFI

# Wed May  3 15:18:18 2023: Log information is stored in file /home/katharina/git/GenomeAnnotation_Workshop2023/BRAKER1/braker.log
#*********
# WARNING: Number of reliable training genes is low (199). Recommended are at least 600 genes
#*********



real	2m37.526s
user	4m27.698s
sys	0m7.730s


<details>
  <summary><b>Out of time, job died? Click here.</b></summary>
This job should easily complete within a few minutes, but you may copy the most important files as follows from a notebook cell:

```
%%script bash
cp -r BRAKER1_precomputed_results BRAKER1
```
</details>

Note that BRAKER by default expects scripts and binaries in a location relative to the `$AUGUSTUS_CONFIG_PATH`. We here changed the location of the `$AUGUSTUS_CONFIG_PATH` to a writable location. Therefore, we have to tell BRAKER where the scripts and binaries are (`--AUGUSTUS_BIN_PATH`, `--AUGUSTUS_CONFIG_PATH`).

The most important output files that we will later use for running TSEBRA are 

   * [BRAKER1/braker.gtf](BRAKER1/braker.gtf)
   * [BRAKER1/Augustus/augustus.hints.gtf](BRAKER1/Augustus/augustus.hints.gtf)
   * [BRAKER1/GeneMark-EP/genemark.gtf](BRAKER1/GeneMark-EP/genemark.gtf)
   * [BRAKER1/hintsfile.gff](BRAKER1/hintsfile.gff)
   
The file [BRAKER1/what-to-cite.txt](BRAKER1/what-to-cite.txt) advises you on what papers should be cited if you were going to publish a manuscript on a gene set produced with BRAKER1.

### TSEBRA

TSEBRA is a tool for selecting a highly accurate gene set from several input sets according to supporting extrinsic evidence. BRAKER internally executes TSEBRA to combine the GeneMark and the AUGUSTUS gene set. If all went well, you do not have run TSEBRA, separately, at all. However, there are several scenarios where TSEBRA may be useful:

   * you are unhappy with the braker.gtf file of any BRAKER version while the files of AUGSTUS and GeneMark look fine. In this case, the supporting evidence was so weak that too many gene models got discarded during the original TSEBRA run within BRAKER. The solution here is to pick one of the gene sets (e.g. AUGUSTUS if that seems to be the better set)
   
   * BRAKER3 failed executing because the provided RNA-Seq data set was too small. In this case, you may wish to combine a BRAKER1 and a BRAKER2 gene set.
   
First, we will have a look at how to generally run TSEBRA on the example of merging the BRAKER1 and BRAKER2 gene set according to the respective evidence of these runs:

In [3]:
%%script bash

mkdir TSEBRA
cd TSEBRA
tsebra.py -g ../BRAKER1/Augustus/augustus.hints.gtf,../BRAKER1/GeneMark-ET/genemark.gtf,../BRAKER2/Augustus/augustus.hints.gtf,../BRAKER2/GeneMark-EP/genemark.gtf \
    -e ../BRAKER1/hintsfile.gff,../BRAKER2/hintsfile.gff -o tsebra.gtf 2> tsebra.log

Check the file [tsebra.log](tsebra.log) for possible errors. The final gene set is in file [tsebra.gtf](tsebra.gtf).

If you applied this to a real full genome dataset, you want to see an improvement in the TSEBRA output compared to all the inputs. One option is to compare BUSCO scores. In a real scenario, with a complete genome, the BUSCO plot should look more like this (sensitivity should increase in the final TSEBRA set):

<img src="busco_ideally.png" alt="BUSCO results (ideally)" width="600"/>

As a second example, we will assume that the braker.gtf file of a BRAKER3 run was suboptimal. BUSCO scores and other statistics indicated that the AUGUSTUS gene set alone was quite good, but you want to merge the GeneMark-ETP gene set on top of the AUGUSTUS gene set (keeping all transcripts from the AUGUSTUS gene set). We can do it like this:

In [4]:
%%script bash

cd BRAKER3
tsebra.py -k Augustus/augustus.hints.gtf -g GeneMark-ETP/genemark.gtf \
    -e hintsfile.gff -o tsebra.gtf 2> tsebra.log

## Data visualization in the UCSC Genome Browser

Visualization of gene structures in context with extrinsic evidence is essential for coming to a decision on whether a gene set "makes sense" or "does not make sense". Typical problems that you may observe in a genome browser include "split genes" (where evidence implies two genes should in fact be a single gene) or "joined genes" (where evidence implies one gene should be split into two genes).

The UCSC Genome Browser ([paper](https://doi.org/10.1101/gr.229102), [resource](https://genome.ucsc.edu/)) is one of the most popular genome browsers. It has the advantage that you do not have to install a browser instance on your own webserver. Instead, you only need to provide a certain data structure with your target data on a webserver. The UCSC Genome Browser servers can display your data from there. The data structures are called "track data hubs" or "assembly hubs" ([paper](https://doi.org/10.1093/bioinformatics/btt637)). 

MakeHub ([paper](https://doi.org/10.1016/j.gpb.2019.05.003), [software](https://github.com/Gaius-Augustus/MakeHub )) is a python script that fully automates the generation of such track data hubs for novel genomes. In the following, we will generate a simple track data hub for the genome sequence that we annotated with BRAKER3 (takes only a few seconds):

In [5]:
%%script bash

T=4 # adjust to number of threads that you booted with

time make_hub.py -e katharina.hoff@uni-greifswald.de \
    --genome /opt/BRAKER/example/genome.fa --long_label "A chunk from the Arabidopsis thaliana genome" \
    --short_label at_chunk  --bam /opt/BRAKER/example/RNAseq.bam --threads ${T} \
    --latin_name "Arabidopsis thaliana" \
    --assembly_version "artifically split custom assembly" \
    --hints BRAKER3/hintsfile.gff --gene_track BRAKER3/braker.gtf BRAKER3

Generating genome 2bit file ./at_chunk/at_chunk/at_chunk.2bit...
Generating chromsome size info file ./at_chunk/at_chunk/at_chunk.chrom.sizes...
Generating variable step wiggle file for GC content ./tmp-at_chunk/at_chunk.gc5Base.wigVarStep.gz...
Generating bigWig file for GC content ./at_chunk/at_chunk/at_chunk.gc5Base.bw...
Generating cytoband track...
Generating bigBed file for ./tmp-at_chunk/at_chunk.cytoband.bed...
Generating softmasking information bed3 file ./tmp-at_chunk/at_chunk.RMsoft.bed3 from genome data (this may take a while)...
Sorting file ./tmp-at_chunk/at_chunk.RMsoft.bed3...
Generating bigBed file for ./tmp-at_chunk/at_chunk.RMsoft.s.bed3...
Generating bigWig RNA-Seq track(s) from BAM...
Converting compressed wig to bigWig
Writing to track db
Generating gene track from file BRAKER3/braker.gtf with label BRAKER3
Generating bigBed file from bigGenePred format for ./tmp-at_chunk/BRAKER3.fixed.bed...
Next step should be writing trackDb entry...
Separating hints into separ


real	0m9.980s
user	0m14.714s
sys	0m3.291s


In [3]:
%%script bash
which bedToBigBed 

You can't perform the suggested `scp` command from the apphub, unless you have privileges on a University of Greifswald webserver. We have therefore copied a prepared hub in advance. The `hub.txt` is available at https://bioinf.uni-greifswald.de/hubs/at_chunk/hub.txt . Remember that link.

In order to visualize your data, go to https://genome.ucsc.edu/ . Click on `My Data` -> `Track Hubs` -> choose the European mirror -> click on `Connected Hubs` and enter the link https://bioinf.uni-greifswald.de/hubs/at_chunk/hub.txt into the text window -> click on `Add Hub`. Congratulations, your Hub is now connected. You should be able to browse something like this: 

<img src="at_chunk.png" alt="UCSC Genome Browser example" width="1000"/>

### How to know which sequences to browse

The long sequences are usually the most interesting to look at. The following command gives you the names of sequences in the order of descending length, you can copy-paste the sequence names into the search window in the UCSC Genome Browser.

In [30]:
%%script bash

N=5 # how many longest sequences would you like to know about

summarizeACGTcontent.pl /opt/BRAKER/example/genome.fa | grep bases | head -${N} | sort -n \
   | perl -ne 'm/(\d+)\s+bases\.\s+(\S+)/; print "$2\t$1\n";'

X1	125040
X2	125100
X3	125100
X4	125100
X5	125100


## How to run BRAKER (and other software) in Docker

If you have a machine on which you have root permissions and Docker, you can run the exact same container as we have been using during this workshop as follows:

```
sudo docker run --rm -it -u root katharinahoff/ceskykrumlov23-notebook:latest bash
```

You can execute all shell commands that we covered in this notebook in that container. Note: you will have to re-copy the `$AUGUSTUS_CONFIG_PATH` and re-install GeneMark-ES/ET/EP. You might find [this tutorial](https://www.tutorialspoint.com/how-to-copy-files-from-host-to-docker-container#:~:text=Another%20way%20to%20copy%20files,folder%20in%20the%20host%20machine) helpful.

## How to run BRAKER, GALBA, MakeHub, etc. in Singularity

Please read instructions in the [README.md](README.md) file.

## Troubleshooting

### I have 80.000 genes predicted by BRAKER/TSEBRA in a full genome, what shall I do?

Please first check whether you are referring to genes, or to transcripts. BRAKER predicts alternative isoforms. If RNA-Seq data supports this, the number of alternative transcripts may be large, but likely true. If it's really genes that you counted, then 80.000 sounds way too much, indeed. Most likely, GeneMark-ET/ES/EP/ETP produced highly fragmented training genes for AUGUSTUS. This will also lead to highly fragmented genes predicted by AUGUSTUS. First, check whether you genome has been masked for repeats. Consider using the additional TRF masking desribed at the top of this notebook. If that does not help, and if you have a protein set of closely related species at hand, consider using that protein set as sole training data for AUGUSTUS. You can use GALBA for this (https://github.com/Gaius-Augustus/GALBA).

### I have only 10.000 genes predicted by BRAKER/TSEBRA in a full genome, what shall I do?

Check whether the BRAKER output files in subfolders Augustus and GeneMark-* produced more genes than TSEBRA. By default, TSEBRA will discard genes without evidence. If you have only little evidence for your species, TSEBRA might be a bad idea. You can also try rerunning TSEBRA with enforcing one of the gene sets.

### How do I know how many genes to expect?

Hard to say. You can download gene sets of related species e.g. from NCBI Genomes, and count. Some gene sets tend to be "underannotated", i.e. they may represent rather the lower numbers of what might be realistic. Katharina usually gets nervous about more than 45000 genes and fewer than 15000 genes. These are definitely weird gene counts. Otherwise: always inspect your gene set in a Genome Browser such as the UCSC Genome Browser to identify problems.

### I have long isoseq RNA-Seq transcripts, can I put them into BRAKER?

No. But we have [other instructions](https://github.com/Gaius-Augustus/BRAKER/blob/master/docs/long_reads/long_read_protocol.md) for you. Please note: isoseq data does not always aid structural genome annotation over short read RNA-Seq data.

### The BUSCO scores of my genome are higher than those of my protein, why?

Some short housekeeping genes are commonly masked during repeat masking. They won't show up in a BRAKER gene set. You can manually add them from BUSCO output to a final gene set. We currently do not provide an automated workflow for this.

### I opened an issue on GitHub about BRAKER or TSEBRA 100 days ago, nobody replied, why?

We are a small team of developers. We try our best and usually respond to well described and easy-to-solve issues within a rather short time frame. Solving other issues may take considerable amounts of time that we simply do not have, or they may be described in a way that we don't know what do with them... please be patient with us.

### I have a problem, whom do I tell?

Please read through the Issues on Github. If the issue does not exist, yet, open an issue.

# Ready to move on?

If you feel confident about your skills, take them to the next level. We have prepared an entire small genome in the following notebook: [Annotate_Hymenolepsis_microstoma.py](Annotate_Hymenolepsis_microstoma.py). We highly recommend that you create a group with several peers in order to reduce runtime (i.e. distribute tasks to people).

### The End